In [4]:
import numpy as np

seq_list = {}
f_fa = open('insulin_pep.uniprot.fa','r')
f_txt = open('BLOSUM62.txt','r')

#load and make dictionary
for line in f_fa:
    if line.startswith('>'):
        seq_h = line.strip()
        seq_list[seq_h] = ''
    else:
        seq_list[seq_h] += line.strip()
f_fa.close()
human_insulin = seq_list['>sp|P01308|INS_HUMAN Insulin OS=Homo sapiens GN=INS PE=1 SV=1']

#BLOSUM62 SCORE MATRIX
for line in f_txt:
    if line.startswith('#'):
        continue
    else:
        splited = line.split()
        break

tmp_prot = ''
for prot in splited:
    tmp_prot += prot
    
prot_score = {}
line_num = 0
for line in f_txt:
    tmp_line = line.split()
    for j in range(1,len(tmp_line)):
        compare = tmp_prot[line_num] + tmp_prot[j-1]
        prot_score[compare] = float(tmp_line[j])
    line_num+=1
    
#start scoring
for key,value in seq_list.items():
    scoring = np.zeros([len(human_insulin)+1,len(value)+1])
    tmp_position = np.zeros([len(human_insulin),len(value)])
    tmp_seq = ''
    os = key.split()[2:4]
    name = os[0][3:] + ' ' + os[1]
    gap_penalty = 11 # controling gap penalty >> -4 is too low penalty
    if key == '>sp|P01308|INS_HUMAN Insulin OS=Homo sapiens GN=INS PE=1 SV=1':
        continue
        
    #calculating score
    for i in range(len(human_insulin)+1):
        scoring[i,0] = -gap_penalty*i
    for j in range(len(value)+1):
        scoring[0,j] = -gap_penalty*j
    for i in range(1,len(human_insulin)+1):
        for j in range(1,len(value)+1):
            tmp_compare = human_insulin[i-1] + value[j-1]
            tmp_score = scoring[i-1,j-1]+ prot_score[tmp_compare]
            scoring[i,j] = max(tmp_score,scoring[i,j-1]-gap_penalty,scoring[i-1,j]-gap_penalty)
            if scoring[i,j] == tmp_score:
                tmp_position[i-1,j-1] = 1
            elif scoring[i,j] == scoring[i,j-1]-gap_penalty: #only care when there are no gaps in human seq
                tmp_position[i-1,j-1] = 1
            else:
                tmp_position[i-1,j-1] = 2
    last = int(j)
    
    #back tracking
    hgap = 0
    for k in range(len(human_insulin)):
        back_1 = len(human_insulin) - k - 1
        if tmp_position[back_1,j-1] == 1:
            tmp_seq += value[j-1]
            j = j-1
        else:
            tmp_seq += '-'
    result_seq = tmp_seq[::-1]
    #comparing sequence
    match = ''
    for a in range(len(human_insulin)):
        if human_insulin[a] == result_seq[a]:
            match += '|'
        else:
            match += ' '
    print name,scoring[i,last]
    print result_seq
    print match
    print human_insulin +'\n'

Mus musculus 431.0
MALLVHFLPLLALLALWEPKPTQAFVKQHLCGPHLVEALYLVCGERGFFYTPKSRREVEDPQVEQLELGGSP--GDLQTLALEVARQKRGIVDQCCTSICSLYQLENYCN
|||    |||||||||| | |  ||| ||||| |||||||||||||||||||| ||| || || | |||| |  | || ||||   |||||| |||||||||||||||||
MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQVGQVELGGGPGAGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN

Xenopus laevis 288.0
MALWMQCLPLVLVLFFSTPN-TEALVNQHLCGSHLVEALYLVCGDRGFFYYPKVKRDMEQALVSGPQDNELDGM-QLQPQEYQ-KM-KRGIVEQCCHSTCSLFQLESYCN
|||||  |||   |    |    | ||||||||||||||||||| ||||| ||  |  |   |         |   |||        ||||||||| | ||| ||| |||
MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQVGQVELGGGPGAGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN

Danio rerio 197.0
MAVWLQAGALLVLLVVSSVSTNPG-TPQHLCGSHLVDALYLVCGPTGFFYNPK-RDVEPLLGFLPPKSAQETEVADFAFKDHAELIRKRGIVEQCCHKPCSIFELQNYCN
|| |     || ||             ||||||||| |||||||  |||| || |     |                          |||||||||   ||   | ||||
MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTP